In [30]:
space

{'max_depth': <hyperopt.pyll.base.Apply at 0x7f7cde8da280>,
 'gamma': <hyperopt.pyll.base.Apply at 0x7f7cddf26250>,
 'reg_alpha': <hyperopt.pyll.base.Apply at 0x7f7cddf26310>,
 'reg_lambda': <hyperopt.pyll.base.Apply at 0x7f7cddf26130>,
 'colsample_bytree': <hyperopt.pyll.base.Apply at 0x7f7cde9d5ee0>,
 'min_child_weight': <hyperopt.pyll.base.Apply at 0x7f7cde9d5f70>,
 'n_estimators': 180,
 'seed': 0,
 'tree_method': 'gpu_hist'}

In [31]:
from dataset_slide import *
import torch
import torch.nn as nn

import numpy as np

from scipy.stats import uniform, randint

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import xgboost as xgb

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score
import sklearn
import logging


def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree'], ), tree_method': 'gpu_hist')

    evaluation = [( X, y)]
    
    clf.fit(X, y,
            eval_set=evaluation, eval_metric="mlogloss",
            early_stopping_rounds=10,verbose=False)


    pred = clf.predict(X)
    f1_score = (pred == y).mean() #sklearn.metrics.f1_score(pred, y, average='macro') #(pred == y).mean()#

    print("f1: {}".format(sklearn.metrics.f1_score(pred, y, average='macro')))

    return {'loss': -f1_score, 'status': STATUS_OK }




SyntaxError: invalid syntax (4042344625.py, line 22)

In [ ]:
person_order = {'F1_Interaction_1': {'P2': 1, 'P1': 1, 'P3': 2},
 'F1_Interaction_2': {'P2': 1, 'P1': 1, 'P3': 2},
 'F2_Interaction_1': {'P4': 1, 'P5': 3},
 'F2_Interaction_2': {'P4': 1},
 'F3_Interaction_1': {'P8': 3, 'P6': 1, 'P7': 1},
 'F3_Interaction_2': {'P6': 1, 'P7': 1},
 'F4_Interaction_1': {'P14': 2,
  'P12': 1,
  'P11': 1,
  'P10': 1,
  'P9': 1,
  'P13': 3},
 'F4_Interaction_2': {'P12': 1,
  'P11': 1,
  'P10': 1,
  'P9': 1,
  'P13': 3},
 'F5_Interaction_1': {'P16': 2, 'P15': 1},
 'F5_Interaction_2': {'P16': 2, 'P15': 1},
 'F6_Interaction_1': {'P19': 3, 'P18': 1, 'P17': 1},
 'F6_Interaction_2': {'P19': 3, 'P18': 1, 'P17': 1},
 'F7_Interaction_1': {'P22': 3,
  'P20': 1,
  'P21': 1,
  'P23': 2},
 'F8_Interaction_1': {'P24': 1, 'P25': 3},
 'F8_Interaction_2': {'P24': 1, 'P25': 3},
 'F8_Interaction_3': {'P24': 1, 'P25': 3},
 'F10_Interaction_1': {'P27': 1, 'P28': 1},
 'F11_Interaction_1': {'P29': 1, 'P30': 2},
 'F11_Interaction_2': {'P29': 1, 'P30': 2},
 'F13_Interaction_1': {'P32': 1, 'P33': 2},
 'F17_Interaction_1': {'P37': 1, 'P38': 2},
 'F17_Interaction_2': {'P37': 1, 'P38': 2}}


group_nums = {1: ['F2_Interaction_2'],
 2: ['F2_Interaction_1',
  'F3_Interaction_2',
  'F5_Interaction_1',
  'F5_Interaction_2',
  'F8_Interaction_1',
  'F8_Interaction_2',
  'F8_Interaction_3',
  'F10_Interaction_1',
  'F11_Interaction_1',
  'F11_Interaction_2',
  'F13_Interaction_1',
  'F17_Interaction_1',
  'F17_Interaction_2'],
 3: ['F1_Interaction_1',
  'F1_Interaction_2',
  'F3_Interaction_1',
  'F6_Interaction_1',
  'F6_Interaction_2'],
 4: ['F7_Interaction_1'],
 5: ['F4_Interaction_2'],
 6: ['F4_Interaction_1']}

group_all_dataset = []
group_ids = group_nums[3]
for group_id in group_ids:
    group_specific_dataset = SpeedDatingDS(group_id = group_id, social_rel = person_order[group_id])
    group_all_dataset.append(group_specific_dataset)

SD = torch.utils.data.ConcatDataset(group_all_dataset)

########################################################################
#Dataloader
########################################################################
train_len = len(SD) - len(SD)//5
test_len = len(SD)//5

for init_seed in [0,1,2]:
    for cv_seed in [0,1,2,3,4]:


        np.random.seed(init_seed)
        index_list = np.arange(len(SD))
        np.random.shuffle(index_list)
        test_range = index_list[list(range(test_len*(cv_seed), test_len*(cv_seed+1)))]
        train_range = index_list[list(set(range(len(SD))) - set(test_range))]
        train = torch.utils.data.Subset(SD, train_range)
        test = torch.utils.data.Subset(SD, test_range)

        batch_size = 32
        trainloader = DataLoader(train, batch_size = train_len, shuffle = True, num_workers = 2)
        testloader = DataLoader(test, batch_size = test_len, shuffle = True, num_workers = 2)



        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        for idx, batch in enumerate(trainloader):

            x_train, vb_output = batch['context'], batch['vb_output']

            labels = vb_output.sum(2).to(device).flatten(start_dim =1)
            index_labels = torch.zeros(x_train.shape[0]).long().to(device)
            index_labels[labels.nonzero()[:,0]] = labels.nonzero()[:,1] + 1 
            y_train = index_labels

        for idx, batch in enumerate(testloader):
            x_test, vb_output = batch['context'], batch['vb_output']

            labels = vb_output.sum(2).to(device).flatten(start_dim =1)
            index_labels = torch.zeros(x_test.shape[0]).long().to(device)
            index_labels[labels.nonzero()[:,0]] = labels.nonzero()[:,1] + 1 
            y_test = index_labels

        X = x_train.flatten(start_dim =1).cpu().numpy()
        y = y_train.cpu().numpy()

        x_test = x_test.flatten(start_dim =1).cpu().numpy()
        y_test = y_test.cpu().numpy()



        space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
                'gamma': hp.uniform ('gamma', 1,9),
                'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
                'reg_lambda' : hp.uniform('reg_lambda', 0,1),
                'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
                'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
                'n_estimators': 180,
                'seed': 0,
                'tree_method': 'gpu_hist'
            }

        trials = Trials()

        tested_best_hyperparams = fmin(fn = objective,
                                space = space,
                                algo = tpe.suggest,
                                max_evals = 10,
                                trials = trials)

        clf = xgb.XGBClassifier(tested_best_hyperparams)

        clf.fit(X, y)
        y_true, y_pred = y_test, clf.predict(x_test)

        f1 = sklearn.metrics.f1_score(y_pred, y_true, average='macro')
        acc = (y_pred == y_true).mean()     
        print(f1)
    #     print("f1: {}".format()))
    #     print("weighted_f1: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='weighted')))
    #     print("acc: {}".format((y_pred == y_true).mean()))
    #     print(confusion_matrix(y_test, y_pred))

        model_unique = "{model}".format(model = "XGBoost_init_seed{}".format(init_seed) + "_cv_seed{}".format(cv_seed))

        clf.save_model("./rebuttal_model_weights/{}.json".format(model_unique))
        
        log_dir = "./rebuttal_logs_fixed_hyperparams/"
        log_path = log_dir + "{model}.log".format(model = model_unique)
        with open(log_path, 'w') as f:
            f.write("NextSpeaker")
            f.write('\n')
            f.write("{}".format(model_unique))
            f.write('\n')
            f.write("acc: {acc}, f1:{f1} ".format(acc = acc, f1 = f1))
            pdb.set_trace()
        
#         logging.basicConfig(filename=log_path)
#         log_statement = "NextSpeaker"
#         logging.warning(log_statement)
#         log_statement = "{}".format(model_unique)
#         logging.warning(log_statement)
#         print(model_unique)
#         print("acc: {acc}, f1:{f1} ".format(acc = acc, f1 = f1))
#         log_statement = "acc: {acc}, f1:{f1} ".format(acc = acc, f1 = f1)
#         logging.warning(log_statement)



  0%|                       | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



f1: 0.23342679751652312                                            
 10%| | 1/10 [00:14<02:06, 14.02s/trial, best loss: -0.875657400450

/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



f1: 0.23342679751652312                                            
 20%|▏| 2/10 [00:29<01:58, 14.79s/trial, best loss: -0.875657400450

...

f1: 0.23342679751652312                                            
 90%|▉| 9/10 [02:06<00:13, 13.87s/trial, best loss: -0.875657400450

/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



f1: 0.23342679751652312                                            
100%|█| 10/10 [02:20<00:00, 14.06s/trial, best loss: -0.87565740045


/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:00:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8271260666806587
> /tmp/ipykernel_8526/4271453457.py(75)<module>()
     73 
     74 for init_seed in [0,1,2]:
---> 75     for cv_seed in [0,1,2,3,4]:
     76 
     77 

ipdb> f1
0.8271260666806587
ipdb> X.shape
(2662, 114)
ipdb> y.Sshape
*** AttributeError: 'numpy.ndarray' object has no attribute 'Sshape'
ipdb> y.shape
(2662,)
ipdb> x_test.shape
(665, 114)
ipdb> y_true
array([0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0

In [7]:
clf = xgb.XGBClassifier(tested_best_hyperparams)

clf.fit(X, y)
y_true, y_pred = y_test, clf.predict(x_test)

f1 = sklearn.metrics.f1_score(y_pred, y_true, average='macro')
acc = (y_pred == y_true).mean()

/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:33:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [8]:
f1

0.8271260666806587

In [4]:
tested_best_hyperparams = {'colsample_bytree': 0.8965848062170019,
 'gamma': 3.6363645213941274,
 'max_depth': 8.0,
 'min_child_weight': 3.0,
 'reg_alpha': 141.0,
 'reg_lambda': 0.7080398610996163}

In [5]:
clf = xgb.XGBClassifier(tested_best_hyperparams)
clf.fit(X, y)
y_true, y_pred = y_test, clf.predict(x_test)

f1 = sklearn.metrics.f1_score(y_pred, y_true, average='macro')
acc = (y_pred == y_true).mean()

/home/don/anaconda3/envs/cuda11/lib/python3.8/site-packages/xgboost/core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


NameError: name 'X' is not defined